# cesm Rainfall Anomalies

In [1]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils._indexDefinitions as _index
import utils.timePeriod as tp

In [2]:
import xarray
import numpy
import cftime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.offsetbox import AnchoredText
import cartopy.io.shapereader as shpr

%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
secondsPerDay = 60*60*24
secondsToTimeP = secondsPerDay*1000 #seconds per day, convert m to mm
#secondsToTimeP = secondsPerDay*_index.nDaysOfInterest["precAnom"]*1000/7 #seconds per month

# Carve out an area of interest, and calculate anomalies

In [5]:
_model.scenarioMip[:,1]

array(['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'BCC-CSM2-MR',
       'CAMS-CSM1-0', 'CAS-ESM2-0', 'CESM2', 'CESM2-WACCM', 'CIESM',
       'CMCC-CM2-SR5', 'CMCC-ESM2', 'CNRM-CM6-1', 'CNRM-CM6-1-HR',
       'CNRM-ESM2-1', 'CanESM5', 'CanESM5-CanOE', 'E3SM-1-1', 'EC-Earth3',
       'EC-Earth3-CC', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR', 'FIO-ESM-2-0',
       'GFDL-CM4', 'GFDL-ESM4', 'GISS-E2-1-G', 'HadGEM3-GC31-LL',
       'HadGEM3-GC31-MM', 'IITM-ESM', 'INM-CM4-8', 'INM-CM5-0',
       'IPSL-CM6A-LR', 'KACE-1-0-G', 'KIOST-ESM', 'MCM-UA-1-0',
       'MIROC-ES2L', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR',
       'MRI-ESM2-0', 'NESM3', 'NorESM2-LM', 'NorESM2-MM', 'UKESM1-0-LL'],
      dtype='<U19')

In [14]:


#model='ACCESS-CM2'
for model in _model.scenarioMip:
    #Calculate a climatology
    #Based on the control run, calculate monthly anomalies
    print(model)
    xr = xarray.merge([
        fh.loadModelData(model[1], 'pr_Amon', 'piControl', model[2]),
        fh.loadModelData(model[1], 'tas_Amon', 'piControl', model[2])
    ])
    monMeansDa=xr.groupby('time.month').mean(dim='time')
    monMeansDa.where(
            (monMeansDa.lat>-50) & (monMeansDa.lat<0) & (monMeansDa.lon>100) & (monMeansDa.lon<170),
            drop=True
        ).load()

    #calculate anomalies for all scenarios
    for experiment in _model.experimentSet:

        #load it
        xr = xarray.merge([
            fh.loadModelData(model[1], 'pr_Amon', experiment, model[3]),
            fh.loadModelData(model[1], 'tas_Amon', experiment, model[3])
        ])
    
        #anom
        anomDa=xr.groupby('time.month')-monMeansDa
        anomDa['model']=model[1]

        #grab area around Australia
        domainDa=anomDa.where(
            (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
            drop=True
        )
    
        domainDa.to_netcdf(
            'results/cmipMonthlyPrTs/'+model[1]+experiment.capitalize()+'.nc')


['CSIRO-ARCCSS' 'ACCESS-CM2' 'r1i1p1f1' 'r1i1p1f1']


KeyboardInterrupt: 

# Time averaging for the anomalies

In [ ]:
for experiment in _model.experimentSet:
    monthlyAnomDa=xarray.concat([
        xarray.open_dataset(
            'results/cmipMonthlyPrTs/'+model+experiment.capitalize()+'.nc') for model in _model.scenarioMip],
    'model')
    

    monthlyAnomDa=monthlyAnomDa.assign({'model':_model.scenarioMip})
    warmSeasonAnomDa=xarray.DataArray()
    
    warmSeasonAnomDa['pr']=secondsToTimeP*tp.averageForTimePeriod(monthlyAnomDa.pr)
    warmSeasonAnomDa['ts']=tp.averageForTimePeriod(monthlyAnomDa.ts)
    warmSeasonAnomDa = warmSeasonAnomDa.assign_attrs([*warmSeasonAnomDa.attrs, ('units','mm/month'), ('timePeriod','Warm Season')])

    warmSeasonAnomDa.to_netcdf('results/cmip'+experiment.capitalize()+'PrecAnoms.nc')